El Filtro de Bayes se basa en el teorema de Bayes para actualizar las creencias o estimaciones de un sistema a medida que se reciben nuevas mediciones. Las ecuaciones fundamentales del Filtro de Bayes son las siguientes:

1. **Predicción:**
   La predicción se realiza utilizando el modelo de transición del estado, que describe cómo evoluciona el estado del sistema con el tiempo. Dado un estado anterior $ x_{t-1} $ y una acción de control $ u_t $, la predicción del estado actual $ x_t $ se da por:
   $$
   p(x_t | u_t, z_{1:t-1}, u_{1:t-1}) = \int p(x_t | x_{t-1}, u_t) p(x_{t-1} | z_{1:t-1}, u_{1:t-1}) dx_{t-1}
   $$

2. **Corrección (Actualización):**
   La corrección se realiza utilizando el teorema de Bayes para incorporar la nueva medición $ z_t $ al estado estimado. La actualización de la creencia se da por:
   $$
   p(x_t | z_{1:t}, u_{1:t}) = \frac{p(z_t | x_t) p(x_t | u_t, z_{1:t-1}, u_{1:t-1})}{p(z_t | z_{1:t-1}, u_{1:t})}
   $$

   Donde:
   - $ p(z_t | x_t) $ es la probabilidad de la medición dada el estado actual, también conocida como modelo de medición o sensor.
   - $ p(x_t | u_t, z_{1:t-1}, u_{1:t-1}) $ es la predicción del estado actual dadas las acciones de control y mediciones anteriores.
   - $ p(z_t | z_{1:t-1}, u_{1:t}) $ es la probabilidad total de la medición actual, y se calcula como:
   $$
   p(z_t | z_{1:t-1}, u_{1:t}) = \int p(z_t | x_t) p(x_t | u_t, z_{1:t-1}, u_{1:t-1}) dx_t
   $$

3. **Estimación del Estado:**
   La estimación del estado $ \hat{x}_t $ se obtiene calculando el valor esperado del estado actual, dado el conjunto de mediciones y acciones de control hasta el tiempo $ t $:
   $$
   \hat{x}_t = \mathbb{E}[x_t | z_{1:t}, u_{1:t}]
   $$

Estas ecuaciones proporcionan un marco general para la implementación del Filtro de Bayes, y se pueden adaptar a diferentes aplicaciones y sistemas mediante la especificación de los modelos de transición y medición.

Vamos a implementar un ejemplo sencillo del Filtro de Bayes para estimar la posición del mouse en una línea (1D). Para simplificar, asumimos que la posición del mouse solo puede estar en una serie de posiciones discretas a lo largo de la línea.

En este ejemplo, usaremos Python y la biblioteca pygame para la interfaz gráfica. Si no tienes instalada la biblioteca pygame, puedes instalarla usando pip:

In [2]:
import pygame
import numpy as np

# Inicializar Pygame
pygame.init()

# Configurar la ventana de visualización
width, height = 800, 100
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Bayesian Filter Mouse Estimation')

# Número de celdas discretas
n_cells = width

# Inicializar la distribución de probabilidad
belief = np.ones(n_cells) / n_cells

# Ruido de sensor
sensor_noise = 20.0

# Bucle principal
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Obtener la posición real del mouse
    mouse_x, _ = pygame.mouse.get_pos()
    
    # Actualizar creencia basada en la medición del sensor
    distances = np.abs(np.arange(n_cells) - mouse_x)
    likelihood = np.exp(-distances**2 / (2 * sensor_noise**2))
    belief = likelihood * belief
    belief /= belief.sum()
    
    # Estimación de la posición
    estimate = np.sum(np.arange(n_cells) * belief)
    
    # Dibujar
    screen.fill((255, 255, 255))
    pygame.draw.circle(screen, (0, 0, 255), (int(mouse_x), height // 2), 10)  # Posición real del mouse
    pygame.draw.circle(screen, (255, 0, 0), (int(estimate), height // 2), 10)  # Estimación del Filtro de Bayes
    pygame.draw.line(screen, (0, 255, 0), (int(estimate), height), (int(estimate), 0), 2)  # Línea de estimación
    pygame.display.flip()
    
    pygame.time.delay(10)

pygame.quit()

En este código, la ventana tiene n_cells celdas, y la creencia se actualiza en cada iteración en función de la posición actual del mouse. La posición estimada del mouse se calcula como la media ponderada de la posición de todas las celdas, y se muestra junto con la posición real del mouse en la ventana de pygame.